In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import datetime as dt

In [0]:
usr = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/data/users.csv')

dvc = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/data/devices.csv')

tr1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/data/transactions_1.csv')
tr2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/data/transactions_2.csv')
tr3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/data/transactions_3.csv')
tr = pd.concat([tr1,tr2,tr3])

ntf = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pd-ds-coh-team1/data/notifications.csv')

In [0]:
tr['created_date']=pd.to_datetime(tr['created_date'])

In [61]:
dict_days_of_transactions = tr.copy()

dict_days_of_transactions['dict_days_of_transactions'] = dict_days_of_transactions['created_date'].dt.date
dict_days_of_transactions = dict_days_of_transactions.groupby(['user_id'])['dict_days_of_transactions'].nunique()
final_dict_days_of_transactions = pd.DataFrame(index=dict_days_of_transactions.axes,data=dict_days_of_transactions.values,columns=['dict_days_of_transactions'])
final_dict_days_of_transactions.head()

,dict_days_of_transactions
user_id,
user_0,161
user_1,71
user_10,63
user_100,25
user_1000,16


In [0]:
usr_dvc = pd.merge(usr,dvc,on='user_id',how='inner')
usr_dvc['age'] = 2020 - usr_dvc['birth_year']

In [0]:
usr_dvc= pd.merge(usr_dvc,final_dict_days_of_transactions,how='left',on='user_id')

In [66]:
usr_dvc.dict_days_of_transactions.isna().sum()

532

In [67]:
def unknown_brand(brand):
  if brand=='Unknown':
    return random.sample(['Android','Apple'],1)[0]
  else:
    return brand

usr_dvc['brand'] = usr_dvc['brand'].apply(unknown_brand)
usr_dvc['brand'].value_counts()

Android    7815
Apple      7729
Name: brand, dtype: int64

In [68]:
devices ={'Android':0 , 'Apple':1}
usr_dvc['brand'] = usr_dvc['brand'].map(devices)

plan = {'STANDARD' : 0, 'SILVER' : 1, 'GOLD' : 1}
usr_dvc['plan'] = usr_dvc['plan'].map(plan)

usr_dvc.shape

(15544, 15)

In [0]:
country = {'GB':'western_europe' , 'FR':'western_europe' , 'IE':'western_europe' , 'DE':'western_europe' , 'CH':'western_europe' , 'NL':'western_europe' ,
           'BE':'western_europe' , 'AT':'western_europe' , 'JE':'western_europe' , 'GG':'western_europe' , 'IM':'western_europe',
           'PL':'central_eastern_europe' , 'RO':'central_eastern_europe' , 'CZ':'central_eastern_europe' , 'HU':'central_eastern_europe' , 'BG':'central_eastern_europe' ,
           'SI':'central_eastern_europe' , 'SK':'central_eastern_europe' , 'HR':'central_eastern_europe' , 'LU':'central_eastern_europe' , 'LI':'central_eastern_europe' ,
           'ES' : 'southern_europe', 'PT': 'southern_europe', 'MT': 'southern_europe', 'IT': 'southern_europe', 'GR': 'southern_europe', 'CY': 'southern_europe', 
           'GI': 'southern_europe','LT' : 'northern_europe', 'LV': 'northern_europe', 'SE': 'northern_europe', 'DK': 'northern_europe', 
           'NO': 'northern_europe', 'FI' : 'northern_europe','EE': 'northern_europe','IS': 'northern_europe', 'GP':'not_europe' ,
           'RE':'not_europe','AU':'not_europe' , 'MQ':'not_europe' }

usr_dvc['country'] = usr_dvc['country'].map(country)

In [70]:
usr_dvc['country'].value_counts()

western_europe            8796
central_eastern_europe    3407
southern_europe           2320
northern_europe            998
not_europe                  23
Name: country, dtype: int64

In [0]:
country_label = usr_dvc['country'].value_counts().index

In [0]:
for i in country_label:
  usr_dvc[i] = np.where(usr_dvc['country']==i,1,0)

In [0]:
tr = tr[(tr['transactions_state'] == 'COMPLETED') & (tr['amount_usd'] > 0)]

In [74]:
tr['transactions_type'].value_counts()

CARD_PAYMENT    999411
TRANSFER        395673
TOPUP           242620
EXCHANGE        124764
ATM              61518
CASHBACK         54499
FEE              18757
CARD_REFUND       9207
TAX               2242
REFUND            1193
Name: transactions_type, dtype: int64

In [75]:
def unknown_brand(brand):
  if brand=='REFUND':
    return random.sample(['CASHBACK','CARD_REFUND'],1)[0]
  else:
    return brand

tr['transactions_type'] = tr['transactions_type'].apply(unknown_brand)
tr['transactions_type'].value_counts()

CARD_PAYMENT    999411
TRANSFER        395673
TOPUP           242620
EXCHANGE        124764
ATM              61518
CASHBACK         55119
FEE              18757
CARD_REFUND       9780
TAX               2242
Name: transactions_type, dtype: int64

In [76]:
types = {'CARD_PAYMENT':'CARD_PAYMENT','TRANSFER':'TRANSFER','TOPUP':'TOPUP','EXCHANGE':'EXCHANGE',
         'ATM':'ATM','CASHBACK':'CASHBACK','CARD_REFUND':'CARD_REFUND',
         'FEE':'FEES','TAX':'FEES'}

tr['transactions_type'] = tr['transactions_type'].map(types)
tr['transactions_type'].value_counts()

CARD_PAYMENT    999411
TRANSFER        395673
TOPUP           242620
EXCHANGE        124764
ATM              61518
CASHBACK         55119
FEES             20999
CARD_REFUND       9780
Name: transactions_type, dtype: int64

In [0]:
tr_amount_usd = tr.groupby(['user_id'], as_index=True)['amount_usd'].agg(['sum','count','mean'])
tr_amount_usd.rename(columns={'sum':'total_amount', 'count':'number_transactions','mean':'avg_amount'}, inplace=True)

In [78]:
tr_amount_usd.head(1)

,total_amount,number_transactions,avg_amount
user_id,,,
user_0,6925.08,520,13.317462


In [0]:
type_encode = pd.concat([pd.get_dummies(tr['transactions_type']),tr],axis=1)

In [80]:
tr_type = type_encode.groupby(['user_id'], as_index=True)['ATM', 'CARD_PAYMENT', 'CARD_REFUND', 'EXCHANGE','CASHBACK', 'FEES', 'TOPUP', 'TRANSFER'].agg('sum').astype(int) 
tr_type.head(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,CASHBACK,FEES,TOPUP,TRANSFER
user_id,,,,,,,,
user_0,0,333,1,5,0,0,158,23
user_1,4,78,0,0,0,0,24,27


In [0]:
usr_dvc = pd.merge(usr_dvc,tr_amount_usd,how='left',on='user_id')

In [0]:
usr_dvc = pd.merge(usr_dvc,tr_type,how='left',on='user_id')

In [83]:
def currency(currency):
  if currency == 'EUR':
    return 'EUR'
  elif currency == 'GBP':
    return 'GBP'
  elif currency == 'PLN':
    return 'PLN'
  elif currency == 'RON':
    return 'RON'
  elif currency == 'USD':
    return 'USD'
  else:
    return 'OTHER'

tr['transactions_currency'] = tr['transactions_currency'].map(currency)
tr['transactions_currency'].value_counts()

EUR      813234
GBP      627338
PLN      154026
OTHER    146022
RON      115960
USD       53304
Name: transactions_currency, dtype: int64

In [84]:
curr_encode = pd.concat([pd.get_dummies(tr['transactions_currency']),tr],axis=1)
count_curr = curr_encode.groupby(['user_id'])['EUR','GBP','PLN','OTHER','RON','USD'].agg('sum')
usr_dvc = pd.merge(usr_dvc,count_curr,how='left',on='user_id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [85]:
usr_dvc.head(2)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age,dict_days_of_transactions,western_europe,central_eastern_europe,southern_europe,northern_europe,not_europe,total_amount,number_transactions,avg_amount,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,CASHBACK,FEES,TOPUP,TRANSFER,EUR,GBP,PLN,OTHER,RON,USD
0,user_0,1992,southern_europe,Madrid,2018-09-10 18:46:42.843855,0,0,1.0,1.0,3,0,0,1,28,161.0,0,0,1,0,0,6925.08,520.0,13.317462,0.0,333.0,1.0,5.0,0.0,0.0,158.0,23.0,25.0,0.0,0.0,495.0,0.0,0.0
1,user_1,1985,northern_europe,Klaipeda,2018-01-11 18:53:10.531146,1,0,NaN,NaN,75,0,0,1,35,71.0,0,0,0,1,0,12955.54,133.0,97.410075,4.0,78.0,0.0,0.0,0.0,0.0,24.0,27.0,133.0,0.0,0.0,0.0,0.0,0.0


In [0]:
ntf = ntf[ntf['status']=='SENT']

In [87]:
reason_encode = pd.concat([pd.get_dummies(ntf['reason'],),ntf],axis=1)
reason_encode.head(2)

,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,ONBOARDING_TIPS_ACTIVATED_USERS,PROMO,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,SILVER_ENGAGEMENT_FEES_SAVED,SILVER_ENGAGEMENT_INACTIVE_CARD,WELCOME_BACK,reason,channel,status,user_id,created_date
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1252,2018-12-02 17:58:33.320645
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_12783,2018-12-01 23:09:37.367127


In [88]:
ntf['reason'].value_counts()

REENGAGEMENT_ACTIVE_FUNDS            22542
PROMO                                13679
NO_INITIAL_CARD_ORDER                 6831
INVEST_IN_GOLD                        5457
NO_INITIAL_CARD_USE                   4406
PROMO_CARD_ORDER                      3685
LOST_CARD_ORDER                       3425
JOINING_ANNIVERSARY                   2733
ONBOARDING_TIPS_ACTIVATED_USERS       2143
BLACK_FRIDAY                          1428
MADE_MONEY_REQUEST_NOT_SPLIT_BILL     1324
ENGAGEMENT_SPLIT_BILL_RESTAURANT      1279
SILVER_ENGAGEMENT_INACTIVE_CARD        787
WELCOME_BACK                           582
METAL_RESERVE_PLAN                     541
BLUE_TUESDAY                           426
SILVER_ENGAGEMENT_FEES_SAVED           425
Name: reason, dtype: int64

In [89]:
count_reason = reason_encode.groupby(['user_id'], as_index=True)['REENGAGEMENT_ACTIVE_FUNDS', 'PROMO', 'NO_INITIAL_CARD_ORDER'
,'INVEST_IN_GOLD', 'NO_INITIAL_CARD_USE', 'PROMO_CARD_ORDER',  'LOST_CARD_ORDER', 'JOINING_ANNIVERSARY', 'ONBOARDING_TIPS_ACTIVATED_USERS', 'BLACK_FRIDAY'
,'MADE_MONEY_REQUEST_NOT_SPLIT_BILL', 'ENGAGEMENT_SPLIT_BILL_RESTAURANT'
,'SILVER_ENGAGEMENT_INACTIVE_CARD', 'WELCOME_BACK', 'METAL_RESERVE_PLAN'
,'BLUE_TUESDAY', 'SILVER_ENGAGEMENT_FEES_SAVED'].agg('sum').astype(int)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [90]:
usr_dvc = pd.merge(usr_dvc,count_reason,how='left',on='user_id')
usr_dvc.head(2)

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age,dict_days_of_transactions,western_europe,central_eastern_europe,southern_europe,northern_europe,not_europe,total_amount,number_transactions,avg_amount,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,CASHBACK,FEES,TOPUP,TRANSFER,EUR,GBP,PLN,OTHER,RON,USD,REENGAGEMENT_ACTIVE_FUNDS,PROMO,NO_INITIAL_CARD_ORDER,INVEST_IN_GOLD,NO_INITIAL_CARD_USE,PROMO_CARD_ORDER,LOST_CARD_ORDER,JOINING_ANNIVERSARY,ONBOARDING_TIPS_ACTIVATED_USERS,BLACK_FRIDAY,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,ENGAGEMENT_SPLIT_BILL_RESTAURANT,SILVER_ENGAGEMENT_INACTIVE_CARD,WELCOME_BACK,METAL_RESERVE_PLAN,BLUE_TUESDAY,SILVER_ENGAGEMENT_FEES_SAVED
0,user_0,1992,southern_europe,Madrid,2018-09-10 18:46:42.843855,0,0,1.0,1.0,3,0,0,1,28,161.0,0,0,1,0,0,6925.08,520.0,13.317462,0.0,333.0,1.0,5.0,0.0,0.0,158.0,23.0,25.0,0.0,0.0,495.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,user_1,1985,northern_europe,Klaipeda,2018-01-11 18:53:10.531146,1,0,NaN,NaN,75,0,0,1,35,71.0,0,0,0,1,0,12955.54,133.0,97.410075,4.0,78.0,0.0,0.0,0.0,0.0,24.0,27.0,133.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [0]:
mask = usr_dvc['attributes_notifications_marketing_push'].isna()
ind =usr_dvc['attributes_notifications_marketing_push'].loc[mask].sample(frac=0.94).index
usr_dvc.loc[ind,'attributes_notifications_marketing_push'] = 1
usr_dvc['attributes_notifications_marketing_push']= usr_dvc['attributes_notifications_marketing_push'].fillna(0)

In [0]:

mask = usr_dvc['attributes_notifications_marketing_email'].isna()
ind = usr_dvc['attributes_notifications_marketing_email'].loc[mask].sample(frac=0.89).index
usr_dvc.loc[ind,'attributes_notifications_marketing_email'] = 1
usr_dvc['attributes_notifications_marketing_email']= usr_dvc['attributes_notifications_marketing_email'].fillna(0)

In [93]:
usr_dvc.describe()

,birth_year,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age,dict_days_of_transactions,western_europe,central_eastern_europe,southern_europe,northern_europe,not_europe,total_amount,number_transactions,avg_amount,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,CASHBACK,FEES,TOPUP,TRANSFER,EUR,GBP,PLN,OTHER,RON,USD,REENGAGEMENT_ACTIVE_FUNDS,PROMO,NO_INITIAL_CARD_ORDER,INVEST_IN_GOLD,NO_INITIAL_CARD_USE,PROMO_CARD_ORDER,LOST_CARD_ORDER,JOINING_ANNIVERSARY,ONBOARDING_TIPS_ACTIVATED_USERS,BLACK_FRIDAY,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,ENGAGEMENT_SPLIT_BILL_RESTAURANT,SILVER_ENGAGEMENT_INACTIVE_CARD,WELCOME_BACK,METAL_RESERVE_PLAN,BLUE_TUESDAY,SILVER_ENGAGEMENT_FEES_SAVED
count,15544.000000,15544.000000,15544.000000,15544.000000,15544.000000,15544.000000,15544.0,15544.0,15544.000000,15544.000000,15012.000000,15544.000000,15544.000000,15544.000000,15544.000000,15544.000000,1.482000e+04,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14820.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000,14740.000000
mean,1983.666624,0.182385,0.073984,0.945445,0.893592,17.874228,0.0,0.0,0.497234,36.333376,47.093392,0.565878,0.219184,0.149254,0.064205,0.001480,8.343306e+03,128.872065,91.935641,4.151012,67.436640,0.659919,8.418623,3.719231,1.416937,16.371120,26.698583,54.874089,42.330499,10.393117,9.853036,7.824561,3.596761,1.529308,0.928019,0.463433,0.370217,0.298915,0.250000,0.232361,0.185414,0.145387,0.096879,0.089824,0.086771,0.053392,0.039484,0.036703,0.028901,0.028833
std,11.369184,0.386174,0.261753,0.227116,0.308369,47.549929,0.0,0.0,0.500008,11.369184,56.646544,0.495657,0.413707,0.356350,0.245125,0.038439,4.345746e+04,222.010712,661.187325,9.632179,117.541926,2.425153,28.188154,32.397735,3.721573,24.881777,76.545989,137.183852,141.758265,57.914892,55.386399,64.153614,23.530306,1.844014,0.936864,0.663332,0.482879,0.657769,0.649296,0.580407,0.388646,0.730512,0.295803,0.285939,0.281508,0.334686,0.194751,0.375766,0.186324,0.246936
min,1929.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,19.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e-02,1.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1977.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.0,0.0,0.000000,28.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.873975e+02,13.000000,15.506063,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1986.000000,0.000000,0.000000,1.000000,1.000000,5.000000,0.0,0.0,0.000000,34.000000,26.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.156395e+03,50.000000,33.019636,1.000000,24.000000,0.000000,0.000000,0.000000,1.000000,8.000000,3.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1992.000000,0.000000,0.000000,1.000000,1.000000,19.000000,0.0,0.0,1.000000,43.000000,64.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.929677e+03,143.250000,65.727010,4.000000,77.000000,0.000000,7.000000,0.000000,1.000000,20.000000,16.000000,44.250000,1

In [0]:
usr_dvc = usr_dvc.fillna(0)

In [95]:
usr_dvc.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brand', 'age',
       'dict_days_of_transactions', 'western_europe', 'central_eastern_europe',
       'southern_europe', 'northern_europe', 'not_europe', 'total_amount',
       'number_transactions', 'avg_amount', 'ATM', 'CARD_PAYMENT',
       'CARD_REFUND', 'EXCHANGE', 'CASHBACK', 'FEES', 'TOPUP', 'TRANSFER',
       'EUR', 'GBP', 'PLN', 'OTHER', 'RON', 'USD', 'REENGAGEMENT_ACTIVE_FUNDS',
       'PROMO', 'NO_INITIAL_CARD_ORDER', 'INVEST_IN_GOLD',
       'NO_INITIAL_CARD_USE', 'PROMO_CARD_ORDER', 'LOST_CARD_ORDER',
       'JOINING_ANNIVERSARY', 'ONBOARDING_TIPS_ACTIVATED_USERS',
       'BLACK_FRIDAY', 'MADE_MONEY_REQUEST_NOT_SPLIT_BILL',
       'ENGAGEMENT_SPLIT_BILL_RESTAURANT', 'SILVER_ENGAGEMENT_INACTIVE

In [0]:
usr_dvc.to_csv('train_set_final.txt',index=False)